In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

import threading
from jetbot import Camera, bgr8_to_jpeg
import cv2
import numpy as np
import tensorflow as tf
import time
import PIL
import io
import math
states = {}
stop_keys = [1,2,3,6] #our class keys
slow_keys = [4]
green_keys = [5]
left_keys = [7]
right_keys = [8]


print("Getting camera")
camera = Camera.instance(width=224, height=224) # to be share by both model
print("Camera instance successful")

prediction = ""

mutex = threading.Lock()
from jetbot import Robot
robot = Robot()

Getting camera
Camera instance successful


In [2]:
#trial figures using area ratio value to determine distance
#arearatio vs probability_threshold
def get_limits(var):
    class_limits = {
                1: (0.01,0.85), #human
                2: (0.05, 0.85), #vehicle
                3: (0.022, 0.7), #red
                4: (0.022, 0.7), #amber
                5: (0.022, 0.7), #green
                6: (0.013, 0.85), #stop
                7: (0.013, 0.85), #left
                8: (0.013, 0.85), #right

    }

    return class_limits.get(var,(0.015,0.8)) #average figure if not found

In [35]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets


imagelink = widgets.Image(format='jpeg', width=300, height=300) #300 for object detection.. road followiung 224 by 224 == camera value default
camera_link = traitlets.dlink((camera, 'value'), (imagelink, 'value'), transform=bgr8_to_jpeg)
display(widgets.HBox([imagelink]))

In [4]:
def changeCarState(info):
    mutex.acquire()
    try:
        global states
        if "None" in str(info):
            states = {}
            overwritten = ""
        else:
            x = info.split(",")

            for i in x:
                y = i.split(":")
                if int(y[0]) in states.keys():
                    states[int(y[0])] = y[1]
                else:
                    states.update({int(y[0]):y[1]})
    finally:
        mutex.release()

def processStateChanged():
    #copy states
    mutex.acquire()
    try:
        if(any([i in states.keys() for i in stop_keys])):  #top priority
            #shifted condition check to obj_detection logic
            #arr = [i in states.keys() for i in stop_keys]
            #print(arr)
            #index_found = arr.index(True)
            #indexkey = stop_keys[index_found]
            #probability = states.get(indexkey, 0.0)
            #if(float(probability) > 0.85): #shifted condition check to obj_detection logic
            robot.left_motor.value = 0.0
            robot.right_motor.value = 0.0
            print("Stop")
        elif (any([i in states.keys() for i in slow_keys])):
            robot.left_motor.value = 0.15 # or value / 2
            robot.right_motor.value = 0.15 # or value /2
            print("Slow Down")
        elif (any([i in states.keys() for i in left_keys])):
            robot.left(0.3)
            print("Turn Left")
        elif (any([i in states.keys() for i in right_keys])):
            robot.right(0.3)
            print("Turn Right")
        elif (any([i in states.keys() for i in green_keys])):
            robot.left_motor.value = 0.7
            robot.right_motor.value = 0.7
            print("Green") #actually do not need to be handled here. just to show i can see green light
    finally:
        mutex.release()


In [5]:
# Creating function to interprete the image and output predictions
modelfilename = "3_refined_cropresize112112.tflite"
category = ["1straight", "2right", "3left"]
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=modelfilename)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [6]:
input_names = ['image_tensor']

def get_frozen_graph(graph_file):
    """Read Frozen Graph file from disk."""
    with tf.gfile.FastGFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

# Create session and load graph
print("Loading TensorRT graph")
start = time.process_time()
# The TensorRT inference graph file downloaded from Colab or your local machine.
pb_fname ="model10000_trt.pb"#opt.graphpb
trt_graph = get_frozen_graph(pb_fname)

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

tf_sess = tf.Session(config=tf_config)

tf.import_graph_def(trt_graph, name='')

tf_input = tf_sess.graph.get_tensor_by_name(input_names[0] + ':0')
tf_scores = tf_sess.graph.get_tensor_by_name('detection_scores:0')
tf_boxes = tf_sess.graph.get_tensor_by_name('detection_boxes:0')
tf_classes = tf_sess.graph.get_tensor_by_name('detection_classes:0')
tf_num_detections = tf_sess.graph.get_tensor_by_name('num_detections:0')

print("Completed loading of TensorRT graph.. took (secs) ",  (time.process_time() - start))

Loading TensorRT graph
Instructions for updating:
Use tf.gfile.GFile.
Completed loading of TensorRT graph.. took (secs)  185.172769072


In [7]:
def non_max_suppression(boxes, probs=None, nms_threshold=0.3):
    """Non-max suppression

    Arguments:
        boxes {np.array} -- a Numpy list of boxes, each one are [x1, y1, x2, y2]
    Keyword arguments
        probs {np.array} -- Probabilities associated with each box. (default: {None})
        nms_threshold {float} -- Overlapping threshold 0~1. (default: {0.3})

    Returns:
        list -- A list of selected box indexes.
    """
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # if the bounding boxes are integers, convert them to floats -- this
    # is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # compute the area of the bounding boxes and grab the indexes to sort
    # (in the case that no probabilities are provided, simply sort on the
    # bottom-left y-coordinate)
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    # if probabilities are provided, sort on them instead
    if probs is not None:
        idxs = probs

    # sort the indexes
    idxs = np.argsort(idxs)

    # keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the index value
        # to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of the bounding
        # box and the smallest (x, y) coordinates for the end of the bounding
        # box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have overlap greater
        # than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last],
                                               np.where(overlap > nms_threshold)[0])))
    # return only the bounding boxes indexes
    return pick

In [34]:
def fixed_positioncheck(change):
    img = change['new'] #224 x 224

    global states

    #overwrites by state manager

    if(len(states) > 0):
        processStateChanged()
        return
   
    #previously
    #img = np.array(img).astype(np.float32)
    #img = img.reshape(1,224,224,3)
    
    # Line Tracing RGB
    if(modelfilename == "3_refined_168168.tflite"):
        img = PIL.Image.open(io.BytesIO(image.value)).resize((168,168))
        img = np.array(img).astype(np.float32)
        img = img.reshape(1,168,168,3)
    elif(modelfilename == "3classes_refined_flipped.tflite"):
        img = np.array(img).astype(np.float32)
        img = img.reshape(1,224,224,3)
    elif(modelfilename == "3_refined_crop164224.tflite"):
        #cropping 
        img = np.array(img).astype(np.float32)
        img_crop = img[60:,:]
        img = img_crop.reshape(1,164,224,3)
    elif(modelfilename == "3_refined_112112.tflite"):
        #test resize for in multiples. other wise use API
        input_size = 224
        output_size = 112
        bin_size = input_size // output_size
        img = img.reshape((output_size, bin_size, output_size, bin_size, 3)).max(3).max(1)
        img = np.array(img).astype(np.float32)
        img = img.reshape(1,112,112,3)
    elif(modelfilename == "3_refined_cropresize112112.tflite"):
        #cropping
        #start = time.process_time()
        img_crop = img[60:,:]
        #resize
        img = PIL.Image.fromarray(img_crop).resize((112,112))
        img = np.array(img).astype(np.float32)
        img = img.reshape(1,112,112,3)
        #print('Preprocessing time : ', time.process_time() - start)    

    start = time.process_time()
    #print(start)

    interpreter.set_tensor(input_details[0]['index'], img)

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    index_predicted = np.argmax(output_data[0])
    global prediction
#     print (index_predicted)
    if category[index_predicted] != prediction:
        #print(category[index_predicted])
        prediction = category[index_predicted]
    if index_predicted == 0:
        #straight
        robot.left_motor.value = 0.5 #* speed_gain_slider.value
        robot.right_motor.value = 0.5 #* speed_gain_slider.value
        #print("To steer STRAIGHT")
    elif index_predicted == 1:
        #right
        #robot.right(0.2 * speed_gain_slider.value)
        robot.left_motor.value = 0.35 #* speed_gain_slider.value
        robot.right_motor.value = 0.15 #* speed_gain_slider.value
        #print("To steer RIGHT")
      
    elif index_predicted == 2:
        #left
        #robot.left(0.2 * speed_gain_slider.value)
        robot.left_motor.value = 0.15 #* speed_gain_slider.value
        robot.right_motor.value = 0.35 #* speed_gain_slider.value
        #print("To steer LEFT")
 
    #print(time.process_time() - start)
    print('Time taken for road following: ', time.process_time() - start)
    time.sleep(0.01)

In [14]:
def performDetection(change):
    img = change['new'] #224 by 224
    global publish_labels
    #image = PIL.Image.fromarray(img) #for debug
    #destination = "./data/test.jpeg" #for debug
    #image.save(destination, "JPEG") #for debug
    #image = cv2.imread(destination) #for debug
    #print(img.shape)

    #300 x 300 from imagelink
    img = PIL.Image.open(io.BytesIO(imagelink.value)).resize((300, 300))
    img = np.array(img).astype(np.float32) #32 ? 16 ? int8 ?
    start = time.process_time()

    scores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections], feed_dict={
        tf_input: img[None, ...]
    })

    boxes = boxes[0] # index by 0 to remove batch dimension
    scores = scores[0]
    classes = classes[0]
    num_detections = int(num_detections[0])

    # Boxes unit in pixels (image coordinates).
    boxes_pixels = []
    for i in range(num_detections):
        # scale box to image coordinates
        #box = boxes[i] * np.array([image.shape[0],
        #                           image.shape[1], image.shape[0], image.shape[1]])
        box = boxes[i] * np.array([img.shape[0],img.shape[0],img.shape[0],img.shape[0]])
        box = np.round(box).astype(int)
        boxes_pixels.append(box)
    boxes_pixels = np.array(boxes_pixels)
    pick = non_max_suppression(boxes_pixels, scores[:num_detections], 0.5)
    if(len(pick) > 0):
        print('Time taken for obj detection: ', time.process_time() - start)  
    publish_labels = ""

    for i in pick:
        box = boxes_pixels[i]
        box = np.round(box).astype(int)
        label = "{}:{:.2f}".format(int(classes[i]), scores[i])
        
        #shifted the decision making to publish or not at the obj_detection logic
        #coordinates used here is box * image width, height, not box = 0.0-1.0
        #object detection uses 300 by 300
        #ratio = area of box / 90000
        #1. retrieve configured values
        #2. calculate box ratio
        #3. compare values and add to publish_labels as when condition met
        limits_val = get_limits(int(classes[i]))
        if(float(scores[i]) > limits_val[1]): #compare configured probability
            ymin = boxes_pixels[i][0]
            xmin = boxes_pixels[i][1]
            ymax = boxes_pixels[i][2]
            xmax = boxes_pixels[i][3]

            height =  math.sqrt((ymax - ymin) * (ymax - ymin)) #no diagonal rect as estimated
            width =  math.sqrt((xmax - xmin) * (xmax - xmin)) #no diagonal rect as estimated
            area_box = height * width
            #300 * 300 = 90000
            if((area_box / 90000) > limits_val[0]): #compare configured ratio
                publish_labels += label +","
        
        
        #print('Box coord: ', box)
        #print('Label: ', label)
        # Draw bounding box.
        #image = cv2.rectangle(
         #   image, (box[1], box[0]), (box[3], box[2]), (0, 
         #                                               255, 0), 2)
        #label = "{}:{:.2f}".format(int(classes[i]), scores[i])
        
    
        # Draw label (class index and probability).
        #draw_label(image, (box[1], box[0]), label) #debug


    # Save and display the labeled image.
    #if(len(pick) > 0):	
    #    save_image(image[:, :, ::-1]) #debug
    #DisplayImage(filename="./data/img.png") #debug

    if(len(pick) > 0):
        if(len(publish_labels) > 0):
            print("publish_labels: " , publish_labels)
            publish_labels = publish_labels[:-1]
            changeCarState(publish_labels)
    else:
        changeCarState("None")

    time.sleep(0.01)

In [13]:
performDetection({'new': camera.value}) #init once

(224, 224, 3)


In [11]:
def roadFollowing():
    #init once
    print("Start road following")
    for x in range(1):
        fixed_positioncheck({'new': camera.value})  # we call the function once to intialize

    robot.left_motor.value = 0.0
    robot.right_motor.value = 0.0

    time.sleep(1) 
    camera.observe(fixed_positioncheck, names='value')

In [12]:
def startObserving():
    print("object detection starts observing!")
    camera.observe(performDetection, names='value')

In [33]:
#reset purpose
camera.unobserve_all()
robot.stop()
#imagelink = widgets.Image(format='jpeg', width=224, height=224)
#camera_link = traitlets.dlink((camera, 'value'), (imagelink, 'value'), transform=bgr8_to_jpeg)
#tf_sess.close()

In [ ]:
#reset purpose
imagelink = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (imagelink, 'value'), transform=bgr8_to_jpeg)

In [36]:
#MAIN running
try:
    tasks = [startObserving,roadFollowing]
    for task in tasks:
        t = threading.Thread(target=task)
        time.sleep(0.5)
        t.start()
except KeyboardInterrupt:
    camera.unobserve_all()
    robot.stop()
    tf_sess.close()
    print("program exited!")

object detection starts observing!
Start road following
Time taken for road following:  0.15642244999980903
Time taken for road following:  0.07138365699984206
Time taken for road following:  0.06235437800000909
Time taken for road following:  0.06130610099990008
Time taken for road following:  0.06461776800006191
Time taken for road following:  0.06304296100006468
Time taken for road following:  0.06979181999986395
Time taken for road following:  0.06874146300015127
Time taken for road following:  0.06848203100003047
Time taken for road following:  0.06029797199994391
Time taken for road following:  0.06381802400005654
Time taken for road following:  0.0637042719999954
Time taken for road following:  0.06304291700007525
Time taken for road following:  0.061947336000002906
Time taken for road following:  0.06756890399992699
Time taken for road following:  0.0667068709999512
Time taken for road following:  0.06819635700003346
Time taken for road following:  0.06374192899988884
Time take

In [ ]:
camera.unobserve_all()
robot.stop()
tf_sess.close()

In [ ]:
startObserving()